In [3]:
from ortools.linear_solver import pywraplp


lp_solver = pywraplp.Solver.CreateSolver("GLOP")
lp_solver

<ortools.linear_solver.pywraplp.Solver; proxy of <Swig Object of type 'operations_research::MPSolver *' at 0x000001E45E3D9F80> >

In [8]:
from numpy import random, zeros, argsort


aggregated_products = 20
production_factors = 10
upper_bound = 100

# production_matrix[i][j] - value of a j-th production factor for i-th product
production_matrix = random.randint(0, upper_bound, size=(aggregated_products, production_factors))

# y[i] - how many first products we want to produce
n1 = 9
y_assigned = [random.randint(1, upper_bound) for i in range(n1)]

# b[i] - recourse variables, the first n1 products are produced with the first production factors and must be greater than y_assigned[i]
b = [random.randint(y_assigned[i] if i < n1 else 0, upper_bound) for i in range(aggregated_products)]

# c - cost of production factors
c = [random.randint(1, upper_bound) for i in range(production_factors)]

# f - penalty for not producing enough of the first n1 products, must be less than c
f = [random.randint(0, c[i]) for i in range(n1)]

# priorities - order of production factors
priorities = zeros(production_factors)

# directive_terms - directive_terms - times we can use the directive term
directive_terms = argsort(random.randint(0, upper_bound, size=production_factors))

In [9]:
y = [lp_solver.IntVar(0, lp_solver.infinity(), f"y{i}") for i in range(aggregated_products)]
z = [lp_solver.IntVar(0, lp_solver.infinity(), f"z{i}") for i in range(production_factors)]

In [10]:
objective = lp_solver.Objective()
for i, priority in enumerate(priorities):
    objective.SetCoefficient(y[i], priority)
objective.SetMaximization()

In [11]:
# Ax <= b
for i in range(aggregated_products):
    constraint = lp_solver.Constraint(-lp_solver.infinity(), b[i])  # y[i] <= b[i]
    for j in range(production_factors):
        constraint.SetCoefficient(y[j], production_matrix[i][j])

TypeError: in method 'Constraint_SetCoefficient', argument 3 of type 'double'